# Calculating DRIS index

In [262]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations

## Data Processing

### Dummy Data

In [263]:
data_diagnosed_dict = {
    'A': [12],
    'Mg': [2],
    'C': [1],
    'D': [3],
    'E': [4]
}

data_optimum_dict = {
    'a': [12,11,10],
    'mg': [1,2,3],
    'c': [1,4,2],
    'd': [1,1,2],
    'e': [2,3,4],
}

df_diagnosed = pd.DataFrame(data_diagnosed_dict)
df_optimum = pd.DataFrame(data_optimum_dict)

### Calculate ratios

In [264]:
def calculate_ratios(df):
    # Step 2: Calculate the ratios
    ratios = {}
    for col1, col2 in combinations(df.columns, 2):
        ratio_name1 = f"{col1}/{col2}"
        # ratio_name2 = f"{col2}/{col1}"
        ratios[ratio_name1] = df[col1] / df[col2]
        # ratios[ratio_name2] = df[col2] / df[col1]

    return pd.DataFrame(ratios)

df_optimum_ratios = calculate_ratios(df_optimum)
df_diagnosed_ratios = calculate_ratios(df_diagnosed)
df_diagnosed_ratios.head()


,A/Mg,A/C,A/D,A/E,Mg/C,Mg/D,Mg/E,C/D,C/E,D/E
0,6.0,12.0,4.0,3.0,2.0,0.666667,0.5,0.333333,0.25,0.75


### Calculate coefficient of variation of optimum values

In [265]:
def calculate_CV(df_ratios):
    cv_dict = {}
    for ratio_name, ratio_values in df_ratios.items():
        mean_ratio = ratio_values.mean()
        std_ratio = ratio_values.std()
        cv = (std_ratio / mean_ratio) * 100  # percentage
        cv_dict[ratio_name] = [cv]
    return pd.DataFrame(cv_dict)

df_optimum_CV = calculate_CV(df_optimum_ratios)
df_optimum_CV.head()

,a/mg,a/c,a/d,a/e,mg/c,mg/d,mg/e,c/d,c/e,d/e
0,64.947979,73.275725,40.563631,43.942507,50.0,33.333333,19.924242,86.60254,61.858957,21.650635


### Calculate rations of the mean of the optimal values

In [266]:
df = pd.DataFrame(df_optimum.mean()).T
df_optimum_mean_ratios = calculate_ratios(df)
df_optimum_mean_ratios.head()

,a/mg,a/c,a/d,a/e,mg/c,mg/d,mg/e,c/d,c/e,d/e
0,5.5,4.714286,8.25,3.666667,0.857143,1.5,0.666667,1.75,0.777778,0.444444


## The function f(A/B) 

\begin{align}
    f(A/B) = 
    \begin{cases}
    &\biggl(\frac{A/B}{a/b} - 1\biggr)\frac{1000}{CV} \quad A/B \ge a/b\\
    &\biggl(1 - \frac{a/b}{A/B}\biggr)\frac{1000}{CV} \quad A/B < a/b
    \end{cases}
\end{align}

- $A/B$ is the ratio of two elements (A and B) in the tissue of the pant being diagnosed.
- $a/b$ is the ration of the optimal values 
- CV is the coefficient of variation of the optimal values $a$ and $b$: $CV = \sigma/\mu$ 

In [267]:
def f(df_diagnosed_ratios, df_optimum_mean_ratios, df_optimum_CV):
    names = df_diagnosed_ratios.columns
    diagnosed_ratios = df_diagnosed_ratios.to_numpy()
    optimum_mean_ratios = df_optimum_mean_ratios.to_numpy()
    optimum_CV = df_optimum_CV.to_numpy()

    def f_single(diagnosed_ratio, optimum_mean_ratio, CV):
        if diagnosed_ratio == 0:
            return float('inf')  # Return inf or some large number to handle zero division
        if diagnosed_ratio >= optimum_mean_ratio:
            f = ((diagnosed_ratio / optimum_mean_ratio) - 1) * 1000 / CV
        else:
            f = (1 - (optimum_mean_ratio / diagnosed_ratio)) * 1000 / CV
        return [f]

    f_dict = dict()
    for i in range(len(names)):
        f_dict[f"f({names[i]})"] = f_single(diagnosed_ratios[0, i], optimum_mean_ratios[0, i], optimum_CV[0, i])
    return pd.DataFrame(f_dict)


# Example usage
# Assuming df_diagnosed_ratios, df_optimum_mean_ratios, df_optimum_CV are defined DataFrames with similar structure
df_f = f(df_diagnosed_ratios, df_optimum_mean_ratios, df_optimum_CV)
df_f.head()


,f(A/Mg),f(A/C),f(A/D),f(A/E),f(Mg/C),f(Mg/D),f(Mg/E),f(C/D),f(C/E),f(D/E)
0,1.399722,21.090949,-26.193414,-5.057113,26.666667,-37.5,-16.730038,-49.074773,-34.127816,31.754265


## Calculate DRIS index


$\mathrm{A_{index}} = \frac{1}{z} f(A/B) + f(A/C) + f(A/D) + \cdots + f(A/N)$

In [268]:
import re

names = df_f.columns.to_numpy()

def create_index(index_letter, names):
    # extracted_pairs = []
    for name in names:
        match = re.search(r'\((\w)/(\w)\)', name)
        if match:
            first_letter = match.group(1)
            second_letter = match.group(2)
            # extracted_pairs.append((first_letter, second_letter))

result = create_index('A', names)

print(result)

None


In [269]:
def create_index_string(index_element, df_diagnosed):
    elements = df_diagnosed.columns
    result_string = f'I_{elements[index_element]} ='
    for i, element in enumerate(elements):
        if index_element < i:
            result_string += (f' + f({elements[index_element]}/{element})')
        elif index_element > i:
            result_string += (f' - f({element}/{elements[index_element]})')
    return result_string

result_string = create_index_string(2,df_diagnosed)
result_string

'I_C = - f(A/C) - f(Mg/C) + f(C/D) + f(C/E)'

In [270]:
def calculate_index_value(index_element, df_diagnosed, df_f):
    f_dict = df_f.to_dict('index')[0]
    elements = df_diagnosed.columns
    result = 0
    for i, element in enumerate(elements):
        if index_element < i:
            result += f_dict[f'f({elements[index_element]}/{element})']
        elif index_element > i:
            result -= f_dict[f'f({element}/{elements[index_element]})']
    return result

calculate_index_value(0,df_diagnosed, df_f)


-8.759857235898371

In [271]:
def calculate_all_index_values(df_diagnosed, df_f):
    elements = df_diagnosed.columns
    results_dict = dict()
    for i in range(len(elements)):
        results_dict[f'I_{elements[i]}'] = [calculate_index_value(i, df_diagnosed, df_f)]
    return pd.DataFrame(results_dict)

DRIS_indices = calculate_all_index_values(df_diagnosed, df_f)

DRIS_indices

,I_A,I_Mg,I_C,I_D,I_E
0,-8.759857,-28.963093,-130.960204,144.522452,24.160702


In [272]:
# save results to CSV 
# DRIS_indices.to_csv('dris.csv', index=False)
DRIS_indices.to_excel('dris.xlsx', index=False)